In [15]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO write the description
'''

datadir = "/datasets/eurora_data/db/"
#datadir = "/datasets/eurora_data/db/"
#datadir = "C:/Users/folid/git-repo/hpc_job_power_prediction/datasets/eurora_data/db_local/"
infile_jobs_to_nodes = datadir + "job_nodes.csv"


infile_nodes = datadir + "nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
#interval_comment = "Andrea"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-06-30')
end_time = pd.to_datetime('2014-11-01')
#infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"

## 4 settings BeforeAlina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-07-01')
infile_jobs = datadir + "CPUs/jobs_cleaned_whole"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']


Along the nodes we have three differentIntel Xeon E5 processors stepping: nodes 1-16 & 25-32 havea maximum frequency of 2.1GHz (CPUs-2100), nodes 17-24 have a maximum frequency of 2.2GHz (CPUs-2200), andnodes 33-64 have a maximum frequency of 3.1GHz (CPUs-3100).

In [16]:
nodes_data = pd.read_csv(infile_nodes, index_col=0)
# nodes_data

### Read job2nodes data

In [17]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


#### solve the problem of jobs runned on node 129

In [18]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129]
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 469020 records


#### solve the problem of duplicates job_id_string, node_id
this is probably caused by the concurrency writing of the database 

In [19]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 444610 records


### Read jobs data

drop the jobs out of interval

In [20]:
jobs_whole = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("jobs_whole_data contains {} records".format(jobs_whole.shape[0]))

jobs_whole_data contains 404792 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
interval_jobs = jobs_whole[pd.to_datetime(jobs_whole['end_time']) <= end_time]
interval_jobs = interval_jobs[pd.to_datetime(interval_jobs['run_start_time']) > start_time]
print("train set contains {} records".format(interval_jobs.shape[0]))

train set contains 100003 records


#### merge

In [22]:
merged_jobs_to_nodes = pd.merge(jobs_to_nodes_whole_data, interval_jobs, on='job_id_string')
print(merged_jobs_to_nodes.shape[0])

108074


In [24]:
#debug
#merged_jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node)]
#merged_jobs_to_node.shape

In [25]:
### remove to continue from a specific job

#long_jobs = long_jobs.drop(['real_pow', 'runned_alone', 'real_pow_quality'], axis=1)

interval_jobs['real_pow'] = 0.0
interval_jobs['ran_alone'] = True
interval_jobs['real_pow_quality'] = 0.0

interval_jobs['n_2_1'] = 0
interval_jobs['n_2_2'] = 0
interval_jobs['n_3_1'] = 0

interval_jobs['job_tot_timepoints'] = 0
interval_jobs['job_timepoints'] = 0
interval_jobs['good_nodes'] = 0 # TODO add this on the file? maybe I can drop it at the end


In [26]:
#debug
#jobs_whole = jobs_whole.head(15)
#debug_merged_jobs_to_nodes = merged_jobs_to_nodes.head(1000)
#merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == 55]['end_time']
#nodes = ['55']

In [27]:
#svi
#jobs_whole[jobs_whole['job_id_string'] == '1006691.node129'].index

In [ ]:
i = 0
outfile = datadir + 'CPUs/' + interval_comment + "/" + interval_comment + "_long_jobs_real_pow_before"

for node in nodes:
    merged_jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node)]
    print("---------- {} jobs in node {}".format(merged_jobs_to_node.shape[0], node))
    infile_node = datadir + "CPUs/" + interval_comment + "/node" + node + suffix + interval_comment + "_active_cores_and_jobs.csv"
    node_measurements = pd.read_csv(infile_node)
    node_start_time = node_measurements['timestamp'].iloc[0]
    num_minutes = node_measurements.shape[0]
    print("-------------------node data starts at {}, num minute {}".format(node_start_time, num_minutes))
    
    for index_job_to_node, row_job_to_node in merged_jobs_to_node.iterrows():
        print()
        print("{}/{}".format(i, merged_jobs_to_nodes.shape[0]))
        
        partial_pow_consumption = 0
        job_id_string = row_job_to_node['job_id_string']
        
        # recupero indice del job in interval_jobs
        job = interval_jobs[interval_jobs['job_id_string'] == job_id_string]
        job_index = interval_jobs[interval_jobs['job_id_string'] == job_id_string].index
        if(job_index != None):
            job_real_consumption = job['real_pow'].iloc[0]
            job_n_2_1 = job['n_2_1'].iloc[0]
            job_n_2_2 = job['n_2_2'].iloc[0]
            job_n_3_1 = job['n_3_1'].iloc[0]

            job_tot_timepoints = job['job_tot_timepoints'].iloc[0]
            job_timepoints = job['job_timepoints'].iloc[0]
            job_good_nodes = job['good_nodes'].iloc[0]
            job_ran_alone = job['ran_alone'].iloc[0]

            node_type = nodes_data.iloc[row_job_to_node['node_id']-1]['cpu_type']
            if(node_type == '2_1_ghz'):
                job_n_2_1 += 1
            elif(node_type == '2_2_ghz'):
                job_n_2_2 += 1
            elif(node_type == '3_1_ghz'):
                job_n_3_1 += 1

            node_idle_core = nodes_data.iloc[row_job_to_node['node_id']-1]['core_idle']
            
            job_start_time = pd.to_datetime(row_job_to_node['run_start_time'])
            job_end_time = pd.to_datetime(row_job_to_node['end_time'])
            
            if(job_end_time - job_start_time >= np.timedelta64(5, 's')):
                before_minutes = int((job_start_time - pd.to_datetime(node_start_time)) / np.timedelta64(5, 's'))
                running_minutes = int((job_end_time - job_start_time) / np.timedelta64(5, 's'))
                after_minutes = num_minutes - running_minutes - before_minutes
                
                before_serie = pd.Series(False, index=np.arange(before_minutes))
                running_serie = pd.Series(True, index=np.arange(running_minutes))
                after_serie = pd.Series(False, index=np.arange(after_minutes))
                concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
                
                interval_measurements = node_measurements[concat_series]
                
                ### take only the entries of the interval where the job was running
                #interval_measurements = node_measurements.loc[(pd.to_datetime(node_measurements['timestamp']) >= pd.to_datetime(row_job_to_node['run_start_time'])) & (pd.to_datetime(node_measurements['timestamp']) <= pd.to_datetime((row_job_to_node['end_time'])) - np.timedelta64(5, 's'))]
                ### drop missing measurements (calculate the percentage)
                n_node_measurements = interval_measurements.shape[0]
                job_tot_timepoints += n_node_measurements

                interval_measurements = interval_measurements.dropna()
                print("{}/{} intervals missing".format(n_node_measurements-interval_measurements.shape[0], n_node_measurements))
                job_timepoints += n_node_measurements-interval_measurements.shape[0]

                ### drop where active cores is greater than 16 - this problem occours only once on node 30
                interval_measurements = interval_measurements[interval_measurements['active_cores'] <= 16]
                ### drop row with 0 active_cores (or less than the current job used cores)
                interval_measurements = interval_measurements[interval_measurements['active_cores'] >= row_job_to_node['ncpus']]  

                if(interval_measurements.shape[0] != 0):
                    job_good_nodes += 1 


                ### group the intervals wehere the partial_pow_cons can be obtaied with the same instance of the formula
                ### take the mean for the pow columns
                # interval_grouped = interval_measurements.reset_index().groupby(["active_cores", "active_jobs", "active_gpus", "active_mics"]).mean()
                interval_grouped = interval_measurements.groupby(["active_cores", "active_jobs", "active_gpus", "active_mics"])
                counts = interval_grouped.size().to_frame(name='counts')
                interval_grouped = (counts
                 .join(interval_grouped.agg({'pow_tot':'mean'}).rename(columns={'pow_tot': 'pow_tot_mean'}))
                 .reset_index()
                )
                interval_grouped.sort_values('active_jobs')

                if(interval_grouped.shape[0]>1):
                    job_ran_alone = False


                if(interval_grouped.shape[0] != 0):
                    #interval_grouped['pow_tot'] = (interval_grouped['pow_tot_0_mean'] + interval_grouped['pow_tot_1_mean']) * row['ncpus'] / interval_grouped['active_cores'] 
                    interval_grouped['pow_tot'] = (interval_grouped['pow_tot_mean'] - (16 - interval_grouped['active_cores'])*node_idle_core) * row_job_to_node['ncpus'] / interval_grouped['active_cores'] 

                    # not_alone_counts = interval_grouped['counts'].loc[interval_grouped['active_jobs'] != 1].sum()
                    # interval_grouped.loc[interval_grouped['active_jobs'] == 1, ['counts']] += not_alone_counts
                    print(interval_grouped)
                    partial_pow_consumption = np.average(interval_grouped['pow_tot'], weights=interval_grouped['counts'])
                    # partial_pow_consumption = partial_pow_consumption / interval_grouped.shape[0]
                    # print(interval_grouped)

                print("partial measurement: {}".format(partial_pow_consumption))
                job_real_consumption += partial_pow_consumption
                partial_pow_consumption = 0

        
            else: # jobs shorter than 5 sec
                   # keep the only timepoint
                measure = node_measurements[node_measurements['timestamp'] == row_job_to_node['run_start_time'][:-3]+':00']
                job_tot_timepoints += 1
                if(measure.shape[0] != 0):
                    job_timepoints += 1
                    job_good_nodes += 1
                    if(measure['active_cores'].iloc[0] != 0):
                        job_ran_alone = False

                    active_cores = measure['active_cores'].iloc[0] + row_job_to_node['ncpus']
                    if(active_cores > 16):
                        active_cores = 16
                    job_real_consumption += (measure['pow_tot'].iloc[0] - (16 - active_cores) * node_idle_core) * row_job_to_node['ncpus'] / active_cores
                else:
                    print("error measure")
                    job_pow_quality = 0.0
               
            
            if(job_real_consumption < 0):
                job_real_consumption = 0.0
            print("job_real_consumption: {}".format(job_real_consumption))
            
            interval_jobs.at[job_index, 'real_pow'] = job_real_consumption
            interval_jobs.at[job_index, 'n_2_1'] = job_n_2_1
            interval_jobs.at[job_index, 'n_2_2'] = job_n_2_2
            interval_jobs.at[job_index, 'n_3_1'] = job_n_3_1
            interval_jobs.at[job_index, 'job_tot_timepoints'] = job_tot_timepoints
            interval_jobs.at[job_index, 'job_timepoints'] = job_timepoints
            interval_jobs.at[job_index, 'good_nodes'] = job_good_nodes
            interval_jobs.at[job_index, 'ran_alone'] = job_ran_alone
    
        
        i = i + 1
    #if(i % 10000 == 0):
    interval_jobs.to_csv(outfile + "_" + node + "_" + str(i) + ".csv")
        
interval_jobs.to_csv(outfile + "_" + node + "_" + str(i) + ".csv")